In [1]:
from cats_dogs_preproc import load_cats_dogs
from cats_dogs_preproc import jpg_to_nparray
import matplotlib.pyplot as plt
%matplotlib inline
import time
import numpy as np
import os
import theano
import theano.tensor as T
import lasagne

<h2>Предобработка</h2>

In [2]:
#загрузка и распаковка датасета
if not os.path.exists("./dogs_vs_cats.train.zip"):
    load_cats_dogs()
if not os.path.exists("./train"):
    os.system("unzip dogs_vs_cats.train.zip > unzip.log")
    
#размер картинки
img_wh = (60,60)
path = ('./train/')
image_names = os.listdir(path)
X,y = jpg_to_nparray(path,image_names,img_wh,grayscale=True)

processed images:  1000
processed images:  2000
processed images:  3000
processed images:  4000
processed images:  5000
processed images:  6000
processed images:  7000
processed images:  8000
processed images:  9000
processed images:  10000
processed images:  11000
processed images:  12000
processed images:  13000
processed images:  14000
processed images:  15000
processed images:  16000
processed images:  17000
processed images:  18000
processed images:  19000
processed images:  20000
processed images:  21000
processed images:  22000
processed images:  23000
processed images:  24000
processed images:  25000


In [3]:
#Нарежем выборку
train_size = (len(X)*4)/5
val_size = len(X)/10
test_size = len(X) - train_size - val_size
X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val = X[train_size: train_size+val_size], y[train_size:train_size+val_size]
X_test, y_test = X[train_size+val_size:], y[train_size+val_size:]


In [4]:
input_X = T.tensor4("X cat/dog image")
target_y = T.vector("target Y cat/dog",dtype='int32')
#размерность входа (None означает "может изменяться")
input_shape = [None] + list(X.shape[1:])

# Save/load

In [5]:
#сохранение
#np.save("catdog_x.npy", X)
#np.save("catdog_y.npy", y)

#загрузка
#X = np.load("catdog_x.npy")
#y = np.load("catdog_y.npy")

# Сборка нейронной сети

In [6]:
#входной слой (вспомогательный)
#first
input_layer = lasagne.layers.InputLayer(shape = input_shape,input_var=input_X)

conv_1 = lasagne.layers.Conv2DLayer(input_layer, num_filters = 10,filter_size = (2,2),
                                    nonlinearity= lasagne.nonlinearities.very_leaky_rectify)
batch_1 = lasagne.layers.batch_norm(conv_1)

pool_1 = lasagne.layers.MaxPool2DLayer(batch_1,pool_size = 5)
batch_2 = lasagne.layers.batch_norm(pool_1)

colv_2 = lasagne.layers.Conv2DLayer(batch_2, num_filters = 10,filter_size = (2,2),
                                    nonlinearity= lasagne.nonlinearities.very_leaky_rectify)
batch_3 = lasagne.layers.batch_norm(colv_2)
pool_2 = lasagne.layers.MaxPool2DLayer(batch_3,pool_size = 5)
batch_4 = lasagne.layers.batch_norm(pool_2)

dense_output = lasagne.layers.DenseLayer(batch_4,num_units = 10, 
                                        nonlinearity = lasagne.nonlinearities.softmax)
dense_output = lasagne.layers.DenseLayer(dense_output,num_units = 2, 
                                        nonlinearity = lasagne.nonlinearities.softmax)



In [7]:
#предсказание нейронки (theano-преобразование)
y_predicted = lasagne.layers.get_output(dense_output)
#все веса нейронки (shared-переменные)
all_weights = lasagne.layers.get_all_params(dense_output,trainable=True)

#функция ошибки - средняя кроссэнтропия
loss = lasagne.objectives.categorical_crossentropy(y_predicted,target_y).mean()

#регуляризация
loss += lasagne.regularization.l2(loss)

accuracy = lasagne.objectives.categorical_accuracy(y_predicted,target_y).mean()

#сразу посчитать словарь обновлённых значений с шагом по градиенту, как раньше
updates_sgd = lasagne.updates.adamax(loss,all_weights,learning_rate = 0.01)

#функция, которая обучает сеть на 1 шаг и возвращащет значение функции потерь и точности
train_fun = theano.function([input_X,target_y],[loss,accuracy],updates= updates_sgd)

#функция, которая считает точность
accuracy_fun = theano.function([input_X,target_y],accuracy)
test =[]
train = []

<h2>Обучение</h2>


In [8]:
#Наша старая знакомая - нарезалка выборки на минибатчи
def iterate_minibatches(inputs, targets, batchsize):
    assert len(inputs) == len(targets)
    indices = np.arange(len(inputs))
    np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        excerpt = indices[start_idx:start_idx + batchsize]
        yield inputs[excerpt], targets[excerpt]

In [9]:
#итерации обучения

num_epochs = 100 #количество проходов по данным

batch_size = 10 #размер мини-батча

for epoch in range(num_epochs):
        # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_acc = 0
    train_batches = 0
    start_time = time.time()
    for batch in iterate_minibatches(X_train, y_train,batch_size):
        inputs, targets = batch
        train_err_batch, train_acc_batch= train_fun(inputs, targets)
        train_err += train_err_batch
        train_acc += train_acc_batch
        train_batches += 1

    # And a full pass over the validation data:
    val_acc = 0
    val_batches = 0
    for batch in iterate_minibatches(X_val, y_val, batch_size):
        inputs, targets = batch
        val_acc += accuracy_fun(inputs, targets)
        val_batches += 1

    
    # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(
        epoch + 1, num_epochs, time.time() - start_time))

    print("  training loss (in-iteration):\t\t{:.6f}".format(train_err / train_batches))
    print("  train accuracy:\t\t{:.2f} %".format(
        train_acc / train_batches * 100))
    train.append(train_acc / train_batches * 100)
    print("  validation accuracy:\t\t{:.2f} %".format(
        val_acc / val_batches * 100))
    test.append(val_acc / val_batches * 100)

Epoch 1 of 100 took 59.245s
  training loss (in-iteration):		0.992865
  train accuracy:		66.95 %
  validation accuracy:		70.32 %
Epoch 2 of 100 took 96.836s
  training loss (in-iteration):		0.886995
  train accuracy:		71.90 %
  validation accuracy:		70.60 %
Epoch 3 of 100 took 62.381s
  training loss (in-iteration):		0.865261
  train accuracy:		72.39 %
  validation accuracy:		72.08 %
Epoch 4 of 100 took 62.385s
  training loss (in-iteration):		0.860733
  train accuracy:		72.95 %
  validation accuracy:		72.56 %
Epoch 5 of 100 took 64.516s
  training loss (in-iteration):		0.841378
  train accuracy:		73.54 %
  validation accuracy:		74.40 %
Epoch 6 of 100 took 49.857s
  training loss (in-iteration):		0.828309
  train accuracy:		73.99 %
  validation accuracy:		72.40 %
Epoch 7 of 100 took 48.697s
  training loss (in-iteration):		0.828639
  train accuracy:		74.07 %
  validation accuracy:		74.24 %
Epoch 8 of 100 took 48.575s
  training loss (in-iteration):		0.810108
  train accuracy:		74.61 %


# Финальное качество

In [10]:
test_acc = 0
test_batches = 0
for batch in iterate_minibatches(X_test, y_test, 500):
    inputs, targets = batch
    acc = accuracy_fun(inputs, targets)
    test_acc += acc
    test_batches += 1
print("Final results:")
print("  test accuracy:\t\t{:.2f} %".format(
    test_acc / test_batches * 100))
#if(max_accuracy<(test_acc / test_batches * 100)):
#    max_accuracy = test_acc / test_batches * 100

Final results:
  test accuracy:		76.76 %


# Coхранение весов

In [13]:
weights = lasagne.layers.get_all_param_values(dense_output)
np.save('../../Save_weight', weights)

# Веселые графики

In [ ]:
plt.plot(train,color='green')
plt.plot(test ,color= 'red')
plt.plot(train_1,color='purple')
plt.plot(test_1)
plt.show()


In [ ]:
plt.plot(train)
plt.plot(test)
plt.show()